# 分類模型
預計使用: 
>  1. SVM
>  2. KNN
>  3. Naive Bayes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq
import math

In [2]:
import warnings
warnings.filterwarnings("ignore")

# KNN + k-fold CV
    
**提供3類的機率，用分3類的機率當作權重算accuracy，進去cv選k**

#### sklearn manual:    
>  1. knn:  
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html 

>  2. k-fold cross validation:      
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html  

>  3. k-fold 的 scoring 指標:      
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [2]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict,cross_validate

### ** todo ** 
1. 要做幾 fold cross-validation?  
2. KNN 的 K 選擇有那些 ??

In [7]:
# k fold cross validation，k 選多少? 
k= 5  # 分五組   (20% 當 validation 差不多?)

# KNN 的 K 選什麼 ??  
K_choice= [11, 13, 17]  #選擇要取幾個 neighbor (亂試幾個 通常會來個range(1, 20)之類ㄉ)

## 提供 3 類的機率

### 分三類的 accuracy 
**原 KNN : 判對 1，判錯 0， accuracy =  sum(判對的1) / total 數量**    
**改 KNN : if 實際的 label 在knn判斷的前 3 名，留下他的機率， if 沒在前 3 名，機率為 0 ，accuracy =  sum(機率) / total 數量**    

**這裡要使用 **改 KNN ** 算出來的 accuracy 來選 K !!!**

### < 分 3 類算 accuracy 的 function >
input: 
- **prob** : 分類模型套件 predict 出的 不同類別的 probabilty matrix (列:每筆資料，欄:不同的label)
- **clf_label** : 要跟什麼 label 比對抓出機率
- **result** : 要不要印出每筆資料前3可能發生的機率，預設是不印 

output: 
- **acc**: 選對的機率當作權重 の accuracy
- **hit_acc**: 3 個中一個就算對 の accuracy

In [11]:
def acc3type(prob, clf_label, result = False):
    
    '''  
    "prob" : sklearn 套件 predict 出的 不同類別的 probabilty matrix (列:每筆資料，欄:不同的label)
    "clf_label" :  用於 與實際label比對抓出機率， ex: 做cv時，是拿 training set 做，所以是trainlabel；做predict時，是拿 testing set 做，所以是testlabel；
    "result" : 要不要印出每筆資料前3可能發生的機率，預設是不印 (做cv時因為只要看accuracy:不印 ； 做predict時想看機率:印)  
    '''
    
    
    p_acc = []
    hit = 0
    
    for l in range(0, len(prob)):

        # 第l筆資料的各種類別機率list
        rlist = prob[l].tolist() 

        # 最大3個機率的位置 (即第幾類)
        p3_loc = heapq.nlargest(3, range(len(rlist)), key= rlist.__getitem__)    # 這語法我找好久... r明明超簡單...

        # 最大的3類 & 對應的機率
        want = [(label[r], round(rlist[r], 3)) for r in p3_loc if rlist[r] != 0]   #給出 (label, p) 的 tuple 且 p = 0 的話我不要
        want.sort(key = lambda x: x[1], reverse= True)  #讓他按照 p 的大小排序
        
        if result:
            print("第", l, "筆資料，最可能發生的 3 種 (label, p): ", want)  # 從第 0 筆開始哦，之後可看看要不要+1 (怕混淆先沒加) 

            
        pa = [p for (i, p) in want if i == clf_label[l]]  # 挑出與實際label相同的機率。實際label有在model判斷的前三名才有機率，沒有的話給空的list
        p_acc = p_acc + pa
        
        if len(pa)!=0:
            hit = hit + 1
            

    acc = sum(p_acc)/len(prob) # 因為實際label可能沒被判成前3名，會給空字串，so要把他當作0算，去除以全部的資料數量，而不是 p_acc 的數量  
    hit_acc = hit/len(prob)
    
    
    if result:
        print("\n")
        print("選對的機率當作權重 の accuracy :" , round(acc, 3))
        print("3 個中一個就算對 の accuracy :" , round(hit_acc, 3))
        
        '''
        利用"acc"，「選對的機率當作權重 の accuracy」來調參數，因為它會影響表情符號出現的機率，也就是會影響推薦順序
        利用"hit_acc"，「3 個中一個就算對 の accuracy」來呈現，因為大家覺得3個有一個是想要的就很棒ㄌ
        '''
        
    
    return acc, hit_acc

### 1. 選定 * 改KNN * 的 K : 使用 Cross Validation 評估，儲存 accuracy 最大的 model

In [26]:
%%time
# 寫進迴圈檢查用什麼 K 做 KNN 的 accuracy 會最大
#K_accuracy = []

ac= 0 #預設
hit_ac = 0

for i in K_choice:
    
    # knn model
    model = KNeighborsClassifier(n_neighbors = i) 
    
    # k-fold Cross Validation
    kf = KFold(n_splits=k, random_state=1, shuffle=True) # random_state = set.seed()
    prob = cross_val_predict(model, train, trainlabel, cv= kf, n_jobs=-1, method = 'predict_proba')
    
    acc_result = acc3type(prob, trainlabel, result = False)
    accuracy = acc_result[0]
    #K_accuracy.append(round(accuracy, 3))
    
    
    # 若 ac 比較大才儲存 model
    if(accuracy > ac):
        knn_model = model
        ac = accuracy
        hit_ac = acc_result[1]
        K = i

    print("跑完 K = {}, accuracy = {}".format(i, round(accuracy, 3)))
    

    
print(" ")    
print("已儲存 accuracy 最大的 model 為 **knn_model**")
print("{} nn , 選對的機率當作權重 の accuracy 為 {}".format(K, round(ac, 3)))
print("{} nn , 三個中一個就算對 の accuracy 為 {}".format(K, round(hit_ac, 3)))
print(" ")
#print("K =", K_choice,"的 accuracy 分別為:", K_accuracy)

跑完 K = 11, accuracy = 0.859
跑完 K = 13, accuracy = 0.847
跑完 K = 17, accuracy = 0.83
 
已儲存 accuracy 最大的 model 為 **knn_model**
11 nn , 選對的機率當作權重 の accuracy 為 0.859
11 nn , 三個中一個就算對 の accuracy 為 0.985
 
Wall time: 636 ms


### 2. 用決定好的 KNN model 做 testing data
注意! 這裡的 testing data 已經知道 label，因為是從training data 偷偷抓出來看的，但實際上是不知道的 !

這是偷吃步偷偷看 accuracy 和檢視分錯的照片!! 實際做的時候看上面 cross validation 的 accuracy 就可以ㄌ，但偷偷看 testing data可以判斷有沒有overfitting

In [27]:
knn_model.fit(train, trainlabel)

# 所有資料被分到每個類別的機率 (matrix)
prob = knn_model.predict_proba(test)  
np.shape(prob)  #有250筆 共5個類別的資料

(250, 5)

In [28]:
acc3type(prob, testlabel, result = True)

第 0 筆資料，最可能發生的 3 種 (label, p):  [(7, 0.545), (2, 0.364), (1, 0.091)]
第 1 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 2 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 3 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 4 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 5 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 6 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 7 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 8 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 9 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 10 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 11 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 12 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 13 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 14 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 15 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 16 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 17 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 18 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 19 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 20 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 21 筆資料，最可能發生的 3 種 (label, p):  [(1, 1.0)]
第 22 筆資料，最可能發生的 

(0.8807000000000001, 0.984)

<hr>

###  遇到的問題記錄一下 (已解決)

1. What does cross_val_predict return? 莫名其妙給一排機率不確定怎麼來ㄉ....          
In each iteration, label of i'th part of data gets predicted. In the end cross_val_predict merges all partially predicted labels and returns them as the final result.
https://stackoverflow.com/questions/41458834/how-is-scikit-learn-cross-val-predict-accuracy-score-calculated


2. 不知道他的shuffle是把原本資料集打亂，再按順序切成5-fold做cross validation，還是沒有把資料集打亂，是random 切 5 fold    
若是第一個，就不再是trainlabel的順序了    
答案是第二個! https://zhuanlan.zhihu.com/p/81673486 ，所以可以按照原來的順序使用 trainlabel

### 小問題 (應該還好吧?): 
若第三名有兩個，只會選 label 排較前面ㄉ當第三名，但我們的emoji label 排序應該會是 sort 過的，給比較熱門的emoji合理ㄅ